In [1]:
from efficientnet_pytorch import EfficientNet

In [4]:
import torch

In [23]:
from __future__ import print_function

import os
import socket
import time
import sys
import subprocess
import numpy as np
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
print("parse_option")

import torch
import torch.optim as optim
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader

from models import model_pool
from models.util import create_model
from models.resnet_language import LangPuller


from dataset.mini_imagenet import ImageNet, MetaImageNet
from dataset.tiered_imagenet import TieredImageNet, MetaTieredImageNet
from dataset.transform_cfg import transforms_options, transforms_list

from util import adjust_learning_rate, create_and_save_embeds, create_and_save_descriptions
from eval.util import accuracy, AverageMeter, validate
from configs import parse_option_supervised
from PIL import Image
from torch.utils.data import Dataset
import os
import pickle
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import re

parse_option


In [26]:
args = lambda x: None
args.n_ways = 5
args.n_shots = 5
args.n_queries = 30
args.data_root = 'data'
args.data_aug = True
args.n_test_runs = 5
args.n_aug_support_samples = 1
args.set_seed = 20
args.continual = True
args.eval_mode = "few-shot-incremental-fine-tune"
args.n_base_support_samples = 1
args.n_base_aug_support_samples = 0
args.linear_bias = False

In [2]:
model = EfficientNet.from_pretrained('efficientnet-b0',num_classes = 100)

Loaded pretrained weights for efficientnet-b0


In [5]:
ckpt = torch.load('./dumped/cubtest/backbones/continual/efficientb0/pretrained_efficientnet0_last.pth')

In [6]:
model._fc.weight

Parameter containing:
tensor([[ 0.0028,  0.0234, -0.0149,  ..., -0.0165,  0.0130,  0.0184],
        [ 0.0084, -0.0155, -0.0251,  ..., -0.0043,  0.0205,  0.0070],
        [-0.0065,  0.0271,  0.0131,  ..., -0.0184, -0.0196, -0.0014],
        ...,
        [ 0.0165,  0.0073, -0.0235,  ...,  0.0237, -0.0212, -0.0052],
        [ 0.0239,  0.0088, -0.0009,  ..., -0.0079, -0.0091,  0.0221],
        [-0.0026, -0.0090, -0.0006,  ..., -0.0229, -0.0160, -0.0257]],
       requires_grad=True)

In [8]:
model._fc.bias = None

In [9]:
model.load_state_dict(ckpt['model'])

<All keys matched successfully>

In [13]:
model(torch.rand(1,3,224,224))

torch.Tensor

In [21]:
ckpt_not_pre= torch.load('./dumped/cubtest/backbones/continual/efficientb0/ckpt_epoch_90.pth')

In [27]:
model = create_model('efficientnet0', 100, args,  dataset='cub200')

assertion is passed


In [28]:
model.load_state_dict(ckpt_not_pre['model'])

<All keys matched successfully>

In [29]:
for name,param in model.named_parameters():
    print("name")
    print(name)
    print("param:")
    print(param)

name
conv1.weight
param:
Parameter containing:
tensor([[[[ 4.7320e-02, -4.8571e-02, -4.4923e-03],
          [-7.7389e-02, -4.2588e-02,  5.4196e-03],
          [-4.7529e-02, -1.1341e-01, -3.7283e-02]],

         [[-1.5574e-01, -1.4496e-01, -1.2935e-01],
          [-1.3366e-01, -2.1301e-01, -7.4952e-02],
          [-1.4931e-01, -1.7099e-01, -8.7402e-02]],

         [[ 4.9584e-02,  4.4959e-02,  7.2807e-02],
          [ 2.9491e-02, -1.1281e-01,  4.0877e-02],
          [-1.9720e-02, -6.7176e-02, -2.5556e-02]]],


        [[[-2.3609e-01, -2.2767e-01, -1.6720e-01],
          [-2.2512e-01, -2.7541e-01, -2.3770e-01],
          [-1.7539e-01, -2.4857e-01, -2.4430e-01]],

         [[-1.0022e-03,  4.3470e-02,  1.1492e-01],
          [ 1.5984e-02, -7.1010e-02, -1.6687e-02],
          [ 7.4066e-02, -6.9993e-02,  2.5466e-02]],

         [[ 1.0369e-01,  4.0135e-02,  1.1944e-01],
          [ 6.3007e-03, -3.1540e-02, -3.0228e-02],
          [ 4.7091e-02,  1.8223e-02, -4.8300e-02]]],


        [[[-2.1946e

In [ ]:
def augment_base_classifier_(net,
                                n,
                                novel_weight=None,
                                novel_bias=None):

    # Create classifier weights for novel classes.
    base_device = self.classifier.weight.device
    base_weight = self.classifier.weight.detach()
    if self.classifier.bias is not None:
        base_bias = self.classifier.bias.detach()
    else:
        base_bias = None

    if novel_weight is None:
        novel_classifier = nn.Linear(base_weight.size(1), n, bias=(base_bias is not None)) # TODO!!
        novel_weight     = novel_classifier.weight.detach()
        if base_bias is not None and novel_bias is None:
            novel_bias = novel_classifier.bias.detach()

    augmented_weight = torch.cat([base_weight, novel_weight.to(base_device)], 0)
    self.classifier.weight = nn.Parameter(augmented_weight, requires_grad=True)

    if base_bias is not None:
        augmented_bias = torch.cat([base_bias, novel_bias.to(base_device)])
        self.classifier.bias = nn.Parameter(augmented_bias, requires_grad=True)

In [45]:
import copy

In [29]:
model.get_image_size('efficientnet-b0')

224

In [99]:
l  = torch.nn.Linear(1280,100,bias = False)

In [ ]:
l.detach()

In [87]:
model_modified._fc.weight.shape

torch.Size([200, 1280])

In [90]:
model_modified._fc.bias = None

In [93]:
model_modified(m).shape

torch.Size([1, 200])

In [49]:
aug_wei = torch.cat([model_modified._fc.weight,l.weight],0)

In [59]:
model_modified.get_image_size('efficientnet-b0')

224

In [96]:
m = torch.rand(5,3,224,224)

In [64]:
model_modified(m)

RuntimeError: The expanded size of the tensor (200) must match the existing size (100) at non-singleton dimension 1.  Target sizes: [1, 200].  Tensor sizes: [100]

In [52]:
model_modified._fc.weight = torch.nn.Parameter(aug_wei,requires_grad = True)

In [46]:
model_modified = copy.deepcopy(model)

In [3]:
import torch

In [15]:
t = torch.nn.Linear(224,100,bias = False)

In [16]:
k = torch.nn.Linear(224,5,bias = False)

In [17]:
t.weight.shape

torch.Size([100, 224])

In [18]:
k.weight.shape

torch.Size([5, 224])

In [19]:
aug_weight = torch.cat([t.weight,k.weight],0)

In [20]:
torch.nn.Linear(224,100,bias = False)
t.weight = torch.nn.Parameter(aug_weight,requires_grad=True)

In [24]:
input = torch.rand(224)

In [26]:
t(input).shape

torch.Size([105])